In [2]:
import torch
import yaml
import ast
#from dig.threedgraph.dataset import QM93D
from dig.threedgraph.dataset import MD17
from dig.threedgraph.dataset.PygTobermorite import Tobermorite
from dig.threedgraph.method import DimeNetPP,SchNet,SphereNet #SchNet, DimeNetPP, ComENet
from dig.threedgraph.method import run
from dig.threedgraph.evaluation import ThreeDEvaluator

# Load configs

In [3]:
with open('config.yaml', 'r') as c:
    config = yaml.safe_load(c)
    
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [4]:
name = config['name']
n_train = config['n_train']
n_val = config['n_val']
seed = config['seed']
energy_and_force = config['energy_and_force']
cutoff = config['cutoff']
num_layers = config['num_layers']
hidden_channels = config['hidden_channels']
out_channels = config['out_channels']
int_emb_size = config['int_emb_size']
basis_emb_size = config['basis_emb_size']
out_emb_channels = config['out_emb_channels']
num_spherical = config['num_spherical']
num_radial = config['num_radial']
envelope_exponent = config['envelope_exponent']
num_before_skip = config['num_before_skip']
num_after_skip = config['num_after_skip']
num_output_layers = config['num_output_layers']

epochs = config['epochs']
batch_size = config['batch_size']
vt_batch_size = config['vt_batch_size']
lr = config['lr']
lr_decay_factor = config['lr_decay_factor']
lr_decay_step_size = config['lr_decay_step_size']

In [5]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

# Dataset

In [6]:
dataset = Tobermorite(root='dataset/', name=name)

split_idx = dataset.get_idx_split(len(dataset.data.y), train_size=n_train, valid_size=n_val, seed=seed)

train_dataset, valid_dataset, test_dataset = dataset[split_idx['train']], dataset[split_idx['valid']], dataset[split_idx['test']]
print('train, validaion, test:', len(train_dataset), len(valid_dataset), len(test_dataset))

train, validaion, test: 950 50 6000


/home/zkm/anaconda3/envs/spherenet/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


# Model

In [6]:
model_dime = DimeNetPP(energy_and_force=energy_and_force, cutoff=cutoff, num_layers=num_layers, 
        hidden_channels=hidden_channels, out_channels=out_channels, int_emb_size=int_emb_size, 
        basis_emb_size=basis_emb_size, out_emb_channels=out_emb_channels, 
        num_spherical=num_spherical, num_radial=num_radial, envelope_exponent=envelope_exponent, 
        num_before_skip=num_before_skip, num_after_skip=num_after_skip, num_output_layers=num_output_layers 
        )

model_sphere = SphereNet(energy_and_force=energy_and_force, cutoff=cutoff, num_layers=num_layers, 
        hidden_channels=hidden_channels, out_channels=out_channels, int_emb_size=int_emb_size, 
        basis_emb_size_dist=8, basis_emb_size_angle=8,
                  basis_emb_size_torsion=8, out_emb_channels=out_emb_channels, 
        num_spherical=num_spherical, num_radial=num_radial, envelope_exponent=envelope_exponent, 
        num_before_skip=num_before_skip, num_after_skip=num_after_skip, num_output_layers=num_output_layers 
        )

model_schnet = SchNet(energy_and_force=energy_and_force, cutoff=cutoff, num_layers=6, 
        hidden_channels=128, out_channels=1,
        num_filters=128, num_gaussians=20 
        )

loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

# FLOPs

## tob14

In [10]:
from torch_geometric.data import DataLoader
from tqdm import tqdm
from thop import profile

dataset = Tobermorite(root='dataset/', name='tob14')

split_idx = dataset.get_idx_split(len(dataset.data.y), train_size=950, valid_size=50, seed=seed)

train_dataset, valid_dataset, test_dataset = dataset[split_idx['train']], dataset[split_idx['valid']], dataset[split_idx['test']]
print('train, validaion, test:', len(train_dataset), len(valid_dataset), len(test_dataset))

train_loader = DataLoader(train_dataset, 4, shuffle=True)
for step, batch_data in enumerate(tqdm(train_loader)):
    print(device)
    batch_data = batch_data.to(device)
    inputs = batch_data
    model = model_schnet.to(device)
    flops,params=profile(model,(inputs, ))
    print(flops,params)
    break

train, validaion, test: 950 50 6000


  0%|                                                             | 0/238 [00:00<?, ?it/s]

cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
674660352.0 419969.0


## tob11

In [21]:
from torch_geometric.data import DataLoader
from tqdm import tqdm
from thop import profile

dataset = Tobermorite(root='dataset/', name='tob11')

split_idx = dataset.get_idx_split(len(dataset.data.y), train_size=950, valid_size=50, seed=seed)

train_dataset, valid_dataset, test_dataset = dataset[split_idx['train']], dataset[split_idx['valid']], dataset[split_idx['test']]
print('train, validaion, test:', len(train_dataset), len(valid_dataset), len(test_dataset))

loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

train_loader = DataLoader(train_dataset, 4, shuffle=True)
for step, batch_data in enumerate(tqdm(train_loader)):
    print(device)
    batch_data = batch_data.to(device)
    inputs = batch_data
    model = model_dime.to(device)
    flops,params=profile(model,(inputs, ))
    print(flops,params)
    break

train, validaion, test: 950 50 6000


  2%|██▊                                                                                                                                  | 5/238 [00:00<00:05, 43.80it/s]

cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3635036800.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3688009984.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3691398272.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3681520768.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3648440320.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3671485184.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3540500992.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3601305600.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3627036800.0 1874176.0


  6%|████████▎                                                                                                                           | 15/238 [00:00<00:04, 45.31it/s]

cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3646200064.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3630194816.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3608015872.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3599044992.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3638970880.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3682642560.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3687725952.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3689535872.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3646137088.0 1874176.0
cuda:0
[INFO] Register count_linear()

...

3658994816.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3600509184.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3677381376.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3620563968.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3587703296.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3731770880.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3664162176.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3718907904.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3647154560.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3576665984.0 1874176.0
cuda:0
[INFO] 

 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊       | 225/238 [00:04<00:00, 46.54it/s]

3683891584.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3537741824.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3704602368.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3573172096.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3559208320.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3632723712.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3630693376.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3609184896.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3683324800.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3604327808.0 1874176.0
cuda:0
[INFO] 

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 238/238 [00:05<00:00, 46.36it/s]

3574840960.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3620752256.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3638132480.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3600907392.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3598824576.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3706611072.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3645927168.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3604500992.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
3622425728.0 1874176.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
1784372352.0 1874176.0


## tob9

In [28]:
from torch_geometric.data import DataLoader
from tqdm import tqdm
from thop import profile

dataset = Tobermorite(root='dataset/', name='tob14')

split_idx = dataset.get_idx_split(len(dataset.data.y), train_size=950, valid_size=50, seed=seed)

train_dataset, valid_dataset, test_dataset = dataset[split_idx['train']], dataset[split_idx['valid']], dataset[split_idx['test']]
print('train, validaion, test:', len(train_dataset), len(valid_dataset), len(test_dataset))

loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

train_loader = DataLoader(train_dataset, 4, shuffle=True)
for step, batch_data in enumerate(tqdm(train_loader)):
    print(device)
    batch_data = batch_data.to(device)
    inputs = batch_data
    model = model_sphere.to(device)
    flops,params=profile(model,(inputs, ))
    print(flops,params)
    break

train, validaion, test: 950 50 6000


  0%|                                                                                                                                             | 0/238 [00:00<?, ?it/s]

cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
4284177536.0 1877952.0
